In [8]:
import os
import sys
import numpy as np
import pandas as pd

notebook_name = 'total_traffic_per_application.ipynb'

current_dir = os.path.dirname(os.path.abspath(notebook_name))
parent_dir = os.path.abspath(os.path.join(current_dir, '..', '..', '..'))


sys.path.insert(1, parent_dir)
from src.data_processer import TelecomDataProcessor


In [2]:
# Replace 'your_conn_params' with your actual connection parameters
conn_params = {
    'host': 'localhost',
    'port': 5432,
    'user': 'ekru',
    'password': 'ekram12345',
    'database': 'tellco',
}

telecom_data_processor = TelecomDataProcessor(conn_params)

In [3]:
df = telecom_data_processor.load_dataset()

/home/ekram/Desktop/week-1/user_analytics_in_the_telecommunication_industry/src/data_processer.py:63: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


In [5]:
cleaned_df = telecom_data_processor.clean_dataset(df)

In [6]:
telecom_data_processor.overview_analysis(cleaned_df)

          BEARER_ID       START_MS         END_MS     DUR._(MS)          IMSI  \
count  1.500010e+05  150001.000000  150001.000000  1.500010e+05  1.500010e+05   
mean   1.013887e+19     499.661213     499.273526  1.046086e+05  2.082016e+14   
std    2.883600e+18     288.201515     287.687241  8.103735e+04  2.144722e+10   
min    6.917538e+18       1.000000       1.000000  7.142000e+03  2.040471e+14   
25%    7.349883e+18     251.000000     252.000000  5.744200e+04  2.082014e+14   
50%    7.349883e+18     499.661213     500.000000  8.639900e+04  2.082015e+14   
75%    1.304243e+19     749.000000     750.000000  1.324300e+05  2.082018e+14   
max    1.318654e+19     999.000000     999.000000  1.859336e+06  2.140743e+14   

       MSISDN/NUMBER          IMEI  AVG_RTT_DL_(MS)  AVG_RTT_UL_(MS)  \
count   1.500010e+05  1.500010e+05    150001.000000    150001.000000   
mean    4.188282e+10  4.847455e+13       109.803795        18.280156   
std     2.438731e+12  2.237359e+13       559.341973   

In [9]:
# Create a new DataFrame for total traffic per application
total_traffic_per_app = df.groupby('MSISDN/NUMBER').agg({
    'SOCIAL_MEDIA_DL_(BYTES)': 'sum',
    'SOCIAL_MEDIA_UL_(BYTES)': 'sum',
    'GOOGLE_DL_(BYTES)': 'sum',
    'GOOGLE_UL_(BYTES)': 'sum',
    'EMAIL_DL_(BYTES)':'sum',
    'EMAIL_UL_(BYTES)':'sum', 
    'YOUTUBE_DL_(BYTES)':'sum', 
    'YOUTUBE_UL_(BYTES)':'sum',
    'NETFLIX_DL_(BYTES)':'sum', 
    'NETFLIX_UL_(BYTES)':'sum', 
    'GAMING_DL_(BYTES)':'sum',
    'GAMING_UL_(BYTES)':'sum',
    'OTHER_DL_(BYTES)':'sum', 
    'OTHER_UL_(BYTES)':'sum',
    
}).reset_index()

melted_df = pd.melt(total_traffic_per_app, id_vars=['MSISDN/NUMBER'], var_name='Application', value_name='Total Traffic (Bytes)')
top_10_per_app = melted_df.groupby('Application').apply(lambda x: x.nlargest(10, 'Total Traffic (Bytes)')).reset_index(drop=True)

print("Top 10 Most Engaged Users per Application:")
print(top_10_per_app)

Top 10 Most Engaged Users per Application:
     MSISDN/NUMBER         Application  Total Traffic (Bytes)
0     4.188282e+10    EMAIL_DL_(BYTES)           1.904048e+09
1     3.361489e+10    EMAIL_DL_(BYTES)           3.360721e+07
2     3.362632e+10    EMAIL_DL_(BYTES)           3.340478e+07
3     3.362578e+10    EMAIL_DL_(BYTES)           3.189541e+07
4     3.378632e+10    EMAIL_DL_(BYTES)           3.087691e+07
..             ...                 ...                    ...
135   3.366716e+10  YOUTUBE_UL_(BYTES)           1.732586e+08
136   3.367588e+10  YOUTUBE_UL_(BYTES)           1.587453e+08
137   3.365826e+10  YOUTUBE_UL_(BYTES)           1.565403e+08
138   3.376127e+10  YOUTUBE_UL_(BYTES)           1.537331e+08
139   3.363004e+10  YOUTUBE_UL_(BYTES)           1.492479e+08

[140 rows x 3 columns]
